## This is a reproduciton & interpretation of http://adventuresinmachinelearning.com/keras-lstm-tutorial/ and https://github.com/adventuresinML/adventures-in-ml-code/blob/master/keras_lstm.py

## Data Prep

In [1]:
import os
path = "/Users/yaoyucui/Works/Git/PTB-dataset-from-Tomas-Mikolov-s-webpage-master/data/"

train_text = open(os.path.join(path+"ptb.train.txt")).read().lower()
test_text = open(os.path.join(path+"ptb.test.txt")).read().lower()
valid_text = open(os.path.join(path+"ptb.valid.txt")).read().lower()

print(train_text[0:200])

 aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memotec mlx nahb punts rake regatta rubens sim snack-food ssangyong swapo wachter 
 pierre <unk> n years o


In [7]:
train_text=train_text.replace("\n","")
train_text=train_text.replace("<unk>","")
test_text=test_text.replace("\n","")
test_text=test_text.replace("<unk>","")
valid_text=valid_text.replace("\n","")
valid_text=valid_text.replace("<unk>","")
   
print(train_text[0:200])

 aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memotec mlx nahb punts rake regatta rubens sim snack-food ssangyong swapo wachter  pierre  n years old wil


In [11]:
from keras.preprocessing.text import text_to_word_sequence
train_list = text_to_word_sequence(train_text)
test_list = text_to_word_sequence(test_text)
valid_list = text_to_word_sequence(valid_text)

train_list[0:10]

['aer',
 'banknote',
 'berlitz',
 'calloway',
 'centrust',
 'cluett',
 'fromstein',
 'gitano',
 'guterman',
 'hydro']

In [60]:
counter = collections.Counter(train_list)
count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))

words, _ = list(zip(*count_pairs))
word_to_id = dict(zip(words, range(len(words))))

train_data = [word_to_id[word] for word in train_list if word in word_to_id]
test_data = [word_to_id[word] for word in test_list if word in word_to_id]
valid_data = [word_to_id[word] for word in valid_list if word in word_to_id]

In [62]:
train_data[:10]

[9619, 9620, 9621, 9623, 9624, 9625, 9628, 9629, 9630, 9631]

## Data Generator

Model first use one batch of (batch size 20 x step size 5 = 100 words) to predict followed them

In [63]:
from keras.utils import to_categorical
import numpy as np
class KerasBatchGenerator(object):

    def __init__(self, data, num_steps, batch_size, vocabulary, skip_step=5):
        self.data = data
        self.num_steps = num_steps
        self.batch_size = batch_size
        self.vocabulary = vocabulary
        # this will track the progress of the batches sequentially through the
        # data set - once the data reaches the end of the data set it will reset
        # back to zero
        self.current_idx = 0
        # skip_step is the number of words which will be skipped before the next
        # batch is skimmed from the data set
        self.skip_step = skip_step
    def generate(self):
        x = np.zeros((self.batch_size, self.num_steps))
        y = np.zeros((self.batch_size, self.num_steps, self.vocabulary))
        while True:
            for i in range(self.batch_size):
                if self.current_idx + self.num_steps >= len(self.data):
                    # reset the index back to the start of the data set
                    self.current_idx = 0
                x[i, :] = self.data[self.current_idx:self.current_idx + self.num_steps]
                temp_y = self.data[self.current_idx + 1:self.current_idx + self.num_steps + 1]
                # convert all of temp_y into a one hot representation
                y[i, :, :] = to_categorical(temp_y, num_classes=self.vocabulary)
                self.current_idx += self.skip_step
            yield x, y

In [87]:
# code expalin:
# default skiptep set to 5, for the first iteration,
# x (in a shape of 20,5) and y (in a shape of 20,5,vocab_size) will be replaced to :
#    x[1,:] 1st word, 2nd word, 3rd word, 4th word, 5th word   y is a one-hot-encoding for 6th - 10th word
# Hence, using the first 5 word to predict next five word.
# And continue from the 6th onward (since skip_num = 5)

batch_size = 20
num_steps = 5
current_idx = 0
skip_step = 5
vocab_size = len(word_to_id)
x = np.zeros((batch_size, num_steps))
y = np.zeros((batch_size, num_steps, vocab_size))

for i in range(batch_size):
    x[i, :] = train_data[current_idx:current_idx + num_steps]
    temp_y = train_data[current_idx + 1:current_idx + num_steps + 1]
    y[i, :, :] = to_categorical(temp_y, num_classes=vocab_size)
    current_idx += skip_step

In [88]:
x

array([[9.619e+03, 9.620e+03, 9.621e+03, 9.623e+03, 9.624e+03],
       [9.625e+03, 9.628e+03, 9.629e+03, 9.630e+03, 9.631e+03],
       [5.126e+03, 9.632e+03, 9.634e+03, 9.635e+03, 9.636e+03],
       [9.637e+03, 9.639e+03, 9.640e+03, 9.641e+03, 9.642e+03],
       [9.643e+03, 9.644e+03, 5.030e+02, 9.645e+03, 9.646e+03],
       [9.647e+03, 8.939e+03, 1.000e+00, 7.300e+01, 3.560e+02],
       [3.000e+01, 2.126e+03, 0.000e+00, 1.460e+02, 1.600e+01],
       [4.000e+00, 8.891e+03, 2.840e+02, 4.180e+02, 1.000e+00],
       [2.000e+01, 1.000e+01, 1.440e+02, 2.000e+00, 1.000e+00],
       [2.501e+03, 0.000e+00, 3.063e+03, 1.600e+03, 9.400e+01],
       [7.461e+03, 1.000e+00, 7.300e+01, 3.560e+02, 6.000e+00],
       [3.410e+02, 1.440e+02, 2.000e+00, 2.461e+03, 6.700e+02],
       [2.160e+03, 9.650e+02, 2.100e+01, 5.310e+02, 4.000e+00],
       [8.891e+03, 2.840e+02, 2.000e+00, 3.700e+01, 3.090e+02],
       [4.500e+02, 3.643e+03, 4.000e+00, 9.510e+02, 2.000e+00],
       [3.129e+03, 5.080e+02, 2.710e+02,

In [92]:
train_data[:20]

[9619,
 9620,
 9621,
 9623,
 9624,
 9625,
 9628,
 9629,
 9630,
 9631,
 5126,
 9632,
 9634,
 9635,
 9636,
 9637,
 9639,
 9640,
 9641,
 9642]

In [94]:
vocab_size = len(word_to_id)
num_steps = 30
batch_size = 200
train_data_generator = KerasBatchGenerator(train_data, num_steps, batch_size, vocabulary = vocab_size,
                                           skip_step=num_steps)
valid_data_generator = KerasBatchGenerator(valid_data, num_steps, batch_size, vocabulary = vocab_size,
                                           skip_step=num_steps)

## Model Structure

Please excuse the initial parameters, I am using a Mac :D

In [95]:
from keras.models import Sequential
from keras.layers import Embedding,LSTM,TimeDistributed,Dropout,Dense,Activation
from keras import optimizers
hidden_size = 30

model = Sequential()
model.add(Embedding(vocab_size, hidden_size, input_length=num_steps))
model.add(LSTM(hidden_size, return_sequences=True))
model.add(LSTM(hidden_size, return_sequences=True))
model.add(Dropout(0.5))
model.add(TimeDistributed(Dense(vocab_size)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 30, 30)            289440    
_________________________________________________________________
lstm_11 (LSTM)               (None, 30, 30)            7320      
_________________________________________________________________
lstm_12 (LSTM)               (None, 30, 30)            7320      
_________________________________________________________________
dropout_6 (Dropout)          (None, 30, 30)            0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 30, 9648)          299088    
_________________________________________________________________
activation_6 (Activation)    (None, 30, 9648)          0         
Total params: 603,168
Trainable params: 603,168
Non-trainable params: 0
_________________________________________________________________


In [96]:
from keras.callbacks import ModelCheckpoint
num_epochs = 5
# checkpointer = ModelCheckpoint(filepath="/Users/yaoyucui/Works/Git/PTB-dataset-from-Tomas-Mikolov-s-webpage-master/data" + '/model-{epoch:02d}.hdf5', verbose=1)
# model.fit_generator(train_data_generator.generate(), len(train_data)//(batch_size*num_steps), num_epochs,
#                         validation_data=valid_data_generator.generate(),
#                         validation_steps=len(valid_data)//(batch_size*num_steps),callbacks=[checkpointer])
model.fit_generator(train_data_generator.generate(), len(train_data)//(batch_size*num_steps),num_epochs)

Epoch 1/5
140/140 [==============================] - 446s 3s/step - loss: 7.8163 - categorical_accuracy: 0.0440
Epoch 2/5
140/140 [==============================] - 425s 3s/step - loss: 6.9377 - categorical_accuracy: 0.0604
Epoch 3/5
140/140 [==============================] - 412s 3s/step - loss: 6.9077 - categorical_accuracy: 0.0603
Epoch 4/5
140/140 [==============================] - 437s 3s/step - loss: 6.8953 - categorical_accuracy: 0.0604
Epoch 5/5
140/140 [==============================] - 416s 3s/step - loss: 6.8898 - categorical_accuracy: 0.0604


#### Result is a lot worse than I expected, this could be caused by:

1. ptb dataset is widly used for ML testing of NLP, there are lots of noise in it.

2. In the original script, words were sorted as its place on a frequency list, there are surely better ways for embedding, this is merely a toy script.

3. hyper-parameters